In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Final4_gurgaon_properties_cleaned(Feature_selection).csv')

In [3]:
df.head()

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,36.0,3.0,2.0,2.0,1.0,850.0,0.0,0.0,0.0,1.0,1.0,0.82
1,0.0,95.0,2.0,2.0,2.0,1.0,1226.0,1.0,0.0,0.0,1.0,2.0,0.95
2,0.0,103.0,2.0,2.0,1.0,1.0,1000.0,0.0,0.0,0.0,1.0,0.0,0.32
3,0.0,99.0,3.0,4.0,4.0,3.0,1615.0,1.0,0.0,1.0,0.0,2.0,1.60
4,0.0,5.0,2.0,2.0,1.0,3.0,582.0,0.0,1.0,0.0,0.0,2.0,0.48


In [4]:
# one hot encode --> sector, balcony, agePossession, furnishing_type, luxury category, floor category

In [5]:
X = df.drop(columns = ['price'])
y = df['price']

In [6]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
import xgboost as xgb

In [7]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [8]:
# Apply log1p transformation to the target variable 
y_transformed = np.log1p(y)

In [9]:
# Creating a column transformer for preprocessing 
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']), #numeric column
        ('cat', OneHotEncoder(drop = 'first'), columns_to_encode)   # category column
    ],
    remainder = 'passthrough'
)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size = 0.2, random_state = 42)

In [66]:
# Creating pipeline 
pipeline = Pipeline([
    ('preprocessor', preprocessor),
   # ('regressor', LinearRegression())
   # ('regressor', Ridge(alpha=1.0))
   # ('regressor', Lasso(alpha=0.1))
   # ('regressor',SVR(kernel = 'rbf'))
   # ('regressor',RandomForestRegressor(n_estimators=100, random_state=42))
   # ('regressor',GradientBoostingRegressor(n_estimators=100, random_state=42))
    ('regressor',xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42))
])

In [67]:
# K-fold cross-validation 
Kfold = KFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_val_score(pipeline, X, y_transformed, cv = Kfold, scoring = 'r2')

In [68]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor',
                 XGBRegressor(b...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [69]:
y_pred = pipeline.predict(X_test)
y_pred_actual = np.expm1(y_pred)
y_test_actual = np.expm1(y_test)

In [70]:
y_pred

array([0.9710937 , 0.5402216 , 0.77362734, 1.8059407 , 1.1252007 ,
       1.8746655 , 2.3065886 , 0.9152702 , 0.9899807 , 0.98393434,
       1.2946134 , 0.7294577 , 1.3548287 , 0.96719646, 2.6425061 ,
       0.8928476 , 1.0989914 , 0.30961433, 0.64699876, 1.197445  ,
       0.86125445, 0.8400612 , 0.3842329 , 0.69261116, 0.9178834 ,
       1.3628646 , 0.6707552 , 0.610745  , 0.8331327 , 2.5584629 ,
       0.31430843, 0.4270928 , 1.6774737 , 0.30835336, 0.78519875,
       1.8063897 , 0.96375436, 0.46531582, 0.5651314 , 0.29446384,
       0.60002446, 0.9710083 , 0.38809618, 0.9981966 , 0.9671493 ,
       0.35811073, 2.216652  , 1.0144746 , 1.1966006 , 0.8424856 ,
       0.8979347 , 0.5740005 , 1.9166346 , 1.1197884 , 0.8082366 ,
       0.7190177 , 2.2082431 , 1.7479016 , 1.7996827 , 0.91272336,
       1.0133266 , 1.2040845 , 0.40564618, 1.5765436 , 0.23365346,
       0.9368657 , 1.1509646 , 1.1431483 , 0.5474792 , 0.75573343,
       1.3300004 , 0.7013671 , 1.0629547 , 1.1672739 , 1.39747

In [71]:
y_pred_actual

array([ 1.6408312 ,  0.71638715,  1.1676147 ,  5.085694  ,  2.0808353 ,
        5.518638  ,  9.040115  ,  1.4974501 ,  1.6911826 ,  1.6749597 ,
        2.6495845 ,  1.0739555 ,  2.876097  ,  1.6305592 , 13.048367  ,
        1.4420738 ,  2.0011375 ,  0.3628994 ,  0.9098004 ,  2.311645  ,
        1.366127  ,  1.3165088 ,  0.46848747,  0.9989282 ,  1.5039849 ,
        2.9073703 ,  0.95571375,  0.8418031 ,  1.3005142 , 11.915949  ,
        0.36931202,  0.53279483,  4.352018  ,  0.36118189,  1.1928427 ,
        5.0884266 ,  1.62152   ,  0.592517  ,  0.7596791 ,  0.34240645,
        0.82216334,  1.6406057 ,  0.4741716 ,  1.7133842 ,  1.6304352 ,
        0.430624  ,  8.176556  ,  1.7579141 ,  2.3088496 ,  1.3221316 ,
        1.4545286 ,  0.7753551 ,  5.7980413 ,  2.064206  ,  1.2439475 ,
        1.0524161 ,  8.099715  ,  4.7425394 ,  5.0477285 ,  1.4910975 ,
        1.7547498 ,  2.3337057 ,  0.5002717 ,  3.838204  ,  0.26320666,
        1.5519702 ,  2.1612408 ,  2.136628  ,  0.72888935,  1.12

In [72]:
y_test_actual

299     1.70
2240    0.78
2384    1.10
3473    5.00
2614    2.65
        ... 
1052    0.71
3275    1.60
1108    1.80
787     0.90
2633    2.00
Name: price, Length: 711, dtype: float64

In [73]:
from sklearn.metrics import r2_score, mean_absolute_error
print(r2_score(y_test_actual, y_pred_actual))
print(mean_absolute_error(y_test_actual, y_pred_actual))

0.8575537772095054
0.4915452009271804


#### Observations:- 
- XGBoost algorithm perform better than rest of the algorithm 